In [15]:
import pandas as pd
import numpy as np

# === Step 1: Load Cleaned Data ===
df = pd.read_csv("sp500_prices_cleaned.csv", index_col=0, parse_dates=True)

# === Step 2: Compute Log Returns ===
log_returns = np.log(df / df.shift(1)).dropna()

# === Step 3: Correlation Matrix ===
correlation_matrix = log_returns.corr()

# === Step 4: Filter Pairs by Correlation Threshold ===
threshold = 0.85  # Adjust this as needed
high_corr_pairs = []

tickers = correlation_matrix.columns
for i in range(len(tickers)):
    for j in range(i + 1, len(tickers)):
        corr_value = correlation_matrix.iloc[i, j]
        if corr_value > threshold:
            high_corr_pairs.append((tickers[i], tickers[j], corr_value))

# === Step 5: Convert to DataFrame and Sort ===
high_corr_df = pd.DataFrame(high_corr_pairs, columns=["Stock A", "Stock B", "Correlation"])
high_corr_df = high_corr_df.sort_values(by="Correlation", ascending=False).reset_index(drop=True)

# === Step 6: Filter out same-company share classes (shared prefixes) ===
def is_share_class_pair(ticker_a, ticker_b):
    return ticker_a[:3] == ticker_b[:3]

# Apply the filter
# Define function to detect same-company share classes
def is_share_class_pair(ticker_a, ticker_b):
    return ticker_a[:3] == ticker_b[:3]

filtered_pairs = []
for _, row in high_corr_df.iterrows():
    a, b = row["Stock A"], row["Stock B"]
    if not is_share_class_pair(a, b):
        filtered_pairs.append(row)

# Create filtered DataFrame
filtered_corr_df = pd.DataFrame(filtered_pairs)

# === Step 8: Show or save results ===
print("Filtered and unique high-correlation pairs (no same-company classes):")
print(filtered_corr_df.head(10))

# Optional: Save
filtered_corr_df.to_csv("high_corr_pairs.csv", index=False)

Filtered and unique high-correlation pairs (no same-company classes):
   Stock A Stock B  Correlation
3      AVB     EQR     0.940642
4     AMAT    LRCX     0.935408
5      DHI     LEN     0.932494
6      MLM     VMC     0.926545
7      EQR     UDR     0.920406
8     AMAT    KLAC     0.920036
9     KLAC    LRCX     0.919107
10     CPT     MAA     0.916288
11     LEN     PHM     0.912155
12     AVB     UDR     0.910825
